In [1]:
"""
1. Теперь решаем задачу регрессии - предскажем цены на недвижимость. 
Использовать датасет https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data (train.csv)
2. Построить случайный лес, вывести важность признаков
3. Обучить стекинг как минимум 3х моделей, использовать хотя бы 1 линейную модель и 1 нелинейную
В качестве решения: Jupyter notebook с кодом, комментариями и графиками
"""

import pandas as pd
import numpy as np
from sklearn.metrics import auc, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor # В классе  DecisionTreeRegressor строится регрессия. Здесь данные, как и
                                            # в DecisionTreeClassifier, делятся на Х и y, только после обучения ожидается 
                                            # значение с плавающей точкой (а не целочисленное, как в DecisionTreeClassifier)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler # Стандартизация функций путем удаления среднего 
                                                  # и масштабирования до единичной дисперсии
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [2]:
"""
1. Решаем задачу линейной регресии 
"""
data = pd.read_csv('train_1.csv')
data


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [3]:
# data.info()
# Считаем кол-во пропусков в каждой колонке:
nan_values = data.isnull().sum().sort_values(ascending=False) 
nan_values.head(20)

# способ удалить пустые столбцы, где записей NaN, например, больше 100 
# v = data.notna().sum().ge(100) # порог >=100 (.ge() означает больше или равно; .lt() - меньше; .gt() - больше; .eq() - равно)
# либо
# v = data.isna().sum().le(100) # в переменной - сумма пустых значений столбца больше или равно 100
# deleted__na_columns = data.drop(v.index[v], axis=1)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
MasVnrType         8
Electrical         1
Id                 0
dtype: int64

In [9]:
# удалим пустые строки с записью NaN
# data_filtred = data.dropna()
data_ = pd.DataFrame.copy(data.drop(nan_values[nan_values>0].index,axis=1)) # axis='columns'== axis=1, а axis='index'== axis=0
print(data_)

        Id  MSSubClass MSZoning  LotArea Street LotShape LandContour  \
0        1          60       RL     8450   Pave      Reg         Lvl   
1        2          20       RL     9600   Pave      Reg         Lvl   
2        3          60       RL    11250   Pave      IR1         Lvl   
3        4          70       RL     9550   Pave      IR1         Lvl   
4        5          60       RL    14260   Pave      IR1         Lvl   
...    ...         ...      ...      ...    ...      ...         ...   
1455  1456          60       RL     7917   Pave      Reg         Lvl   
1456  1457          20       RL    13175   Pave      Reg         Lvl   
1457  1458          70       RL     9042   Pave      Reg         Lvl   
1458  1459          20       RL     9717   Pave      Reg         Lvl   
1459  1460          20       RL     9937   Pave      Reg         Lvl   

     Utilities LotConfig LandSlope  ... EnclosedPorch 3SsnPorch ScreenPorch  \
0       AllPub    Inside       Gtl  ...             0   

In [10]:
#Находим категориальные признаки
cat_feat = list(data_.dtypes[data_.dtypes == object].index) # index - берем построчно 

In [12]:
# Отфильтруем непрерывные признаки
num_feat = [f for f in data_ if f not in (cat_feat + ['Id', 'SalePrice'])]

In [13]:
# Смотрим, сколько у нас значений по каждому категориальному признаку
cat_nunique = data_[cat_feat].nunique()                  # nunique() возвращает количество уникальных объектов (здесь-столбцов)
print(cat_nunique)

MSZoning          5
Street            2
LotShape          4
LandContour       4
Utilities         2
LotConfig         5
LandSlope         3
Neighborhood     25
Condition1        9
Condition2        8
BldgType          5
HouseStyle        8
RoofStyle         6
RoofMatl          8
Exterior1st      15
Exterior2nd      16
ExterQual         4
ExterCond         5
Foundation        6
Heating           6
HeatingQC         5
CentralAir        2
KitchenQual       4
Functional        7
PavedDrive        3
SaleType          9
SaleCondition     6
dtype: int64


In [16]:
# Сравним количество категориальных столбцов с количеством этих столбцов, содержащих уникальные значения 
print(len(cat_nunique)==len(cat_feat))
# print(len(cat_feat))

# # А можно было применить стратегию замены пустых значений столбцов на медианное значение признаков:
# num_feat_median = data.loc[data[data[num_feat].isna()].index, num_feat] = data[num_feat].median() 
# num_feat_median

True


In [17]:
# Объединим 2 вида признаков в одной переменной, чтобы определить ее в Х 

# df = data.reset_index()

factor_feat = pd.concat((data_[num_feat], data_[cat_feat]), axis=1) # объединяем столбцы фрейма data - категориальные
                                                                  # признаки и непрерывные


In [18]:
# Задаем переменные Х и y
X = data_[factor_feat.columns]  
# X = data[data.columns[data.columns!='SalePrice']]   
y = data['SalePrice'] 

In [42]:
# Разбиваем на train/test
D_train, D_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:
print(X.isna().sum().sum())  # проверяем количество ячеек с пустыми значениями в переменной Х 

0


In [24]:
# Создаем дамми-переменные для категорий
dummy_train = pd.get_dummies(D_train[cat_feat], columns=cat_feat)  # преобразовываем категориальную переменную 
dummy_test = pd.get_dummies(D_test[cat_feat], columns=cat_feat)    # в фиктивные / индикаторные переменные

dummy_cols = list(set(dummy_train) & set(dummy_test))           # список уникальных (set) колонок категориальных признаков

dummy_train_q = dummy_train[dummy_cols]                         # обучающийся сет категориальных признаков 
dummy_test_q = dummy_test[dummy_cols]                           # тестовый сет категориальных признаков

In [25]:
# Объединяем 2 вида признаков в Х для обучающей и тестовой выборок 
X_train = pd.concat([D_train[num_feat],
                     dummy_train_q], axis=1)

X_test = pd.concat([D_test[num_feat],
                     dummy_test_q], axis=1)

# indices_to_keep_X_train = ~X_train.isin([np.nan, np.inf, -np.inf]).any(1)
# indices_to_keep_X_test = ~X_test.isin([np.nan, np.inf, -np.inf]).any(1)
# X_train = X_train[indices_to_keep_X_train].astype(np.float64)
# X_test = X_test[indices_to_keep_X_test].astype(np.float64)

In [26]:
# определим модель линейной регресиии и обучим ее 
model = LinearRegression()
model.fit(X_train, y_train) 

# Получим прогнозные значения
y_pred = model.predict(X_test)
# Проверим качество модели 
model.score(X_test, y_test)

0.8724659227902871

In [43]:
"""
2. Строим случайный лес и выводим важность признаков 
"""
# Разбиваем на train/test
# splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# for train_index, test_index in splitter.split(X, y):
#     d_train = X.iloc[train_index]
#     d_test = X.iloc[test_index]
    
#     y_train = data_['SalePrice'].iloc[train_index]
#     y_test = data_['SalePrice'].iloc[test_index]


'\n2. Строим случайный лес и выводим важность признаков \n'

In [46]:
# Разбиваем на train/test
Df_train, Df_test, yf_train, yf_test = train_test_split(X, y, test_size=0.2, random_state=41)


In [47]:
# Создаем дамми-переменные для категорий
dum_train = pd.get_dummies(Df_train[cat_feat], columns=cat_feat)  # преобразовываем категориальную переменную 
dum_test = pd.get_dummies(Df_test[cat_feat], columns=cat_feat)    # в фиктивные / индикаторные переменные

dummy_col = list(set(dum_train) & set(dum_test))           # список уникальных (set) колонок категориальных признаков

dum_train_q = dum_train[dummy_col]                         # обучающийся сет категориальных признаков 
dum_test_q = dum_test[dummy_col]                           # тестовый сет категориальных признаков

In [48]:
# Объединяем 2 вида признаков в Х для обучающей и тестовой выборок 
Xf_train = pd.concat([Df_train[num_feat],
                     dum_train_q], axis=1)

Xf_test = pd.concat([Df_test[num_feat],
                     dum_test_q], axis=1)

In [49]:
# Обучаем решающее дерево
# Немного ограничим глубину и минимальное кол-во объектов в листе для уменьшения переобучения


clf_tree = RandomForestClassifier(max_depth=15, min_samples_leaf=20)
clf_tree.fit(Xf_train, yf_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=20, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [56]:
print('Вес наиболее важного признака: ', max(clf_tree.feature_importances_))
clf_tree.feature_importances_

Вес наиболее важного признака:  0.061491925266229853


array([0.02157648, 0.04655058, 0.03051525, 0.01335716, 0.05471814,
       0.04459719, 0.03893355, 0.00344391, 0.04035526, 0.04502978,
       0.05429799, 0.02829299, 0.        , 0.06149193, 0.0083298 ,
       0.0012003 , 0.01705162, 0.01090299, 0.00947909, 0.00090034,
       0.02531998, 0.01099776, 0.01585673, 0.04399613, 0.02372386,
       0.02703232, 0.00620853, 0.        , 0.00384284, 0.        ,
       0.        , 0.02916135, 0.02433034, 0.        , 0.        ,
       0.        , 0.00790507, 0.        , 0.0035609 , 0.00030757,
       0.        , 0.00165884, 0.        , 0.00373883, 0.        ,
       0.        , 0.00049288, 0.00362543, 0.        , 0.01011535,
       0.00488472, 0.        , 0.00477165, 0.00293482, 0.00170227,
       0.        , 0.00546527, 0.00219035, 0.        , 0.00977144,
       0.        , 0.        , 0.        , 0.00915273, 0.        ,
       0.00205564, 0.00172207, 0.        , 0.        , 0.00061088,
       0.01080383, 0.        , 0.        , 0.00599394, 0.     

In [61]:
imp = pd.Series(clf_tree.feature_importances_)
imp.sort_values(ascending=False)

13     0.061492
4      0.054718
10     0.054298
1      0.046551
9      0.045030
         ...   
118    0.000000
117    0.000000
51     0.000000
113    0.000000
76     0.000000
Length: 178, dtype: float64

In [59]:
# проверим качество модели (низкое)
clf_tree.score(Xf_test, yf_test)

0.0136986301369863

In [84]:
"""
3. Обучить стекинг как минимум 3х моделей, используя хотя бы 1 линейную модель и 1 нелинейную
"""

# Вначале подготовим признаки для обучения 
# Разбиваем на train/test

X = data[data.columns[data.columns!='SalePrice']]   
y = data['SalePrice']

Ds_train, Ds_test, ys_train, ys_test = train_test_split(X, y, test_size=0.3, random_state=39)


In [85]:
# Создаем дамми-переменные для категорий
dummy_s_train = pd.get_dummies(Ds_train[cat_feat], columns=cat_feat)  # преобразовываем категориальную переменную 
dummy_s_test = pd.get_dummies(Ds_test[cat_feat], columns=cat_feat)    # в фиктивные / индикаторные переменные

dummy_s_cols = list(set(dummy_s_train) & set(dummy_s_test))           # список уникальных (set) колонок категориальных признаков

dummy_s_train_q = dummy_s_train[dummy_s_cols]                         # обучающийся сет категориальных признаков 
dummy_s_test_q = dummy_s_test[dummy_s_cols]                           # тестовый сет категориальных признаков]                           # тестовый сет категориальных признаков

In [86]:
# для обучаемого сета находим медианное значение
train_median = Ds_train[num_feat].median()   


In [87]:
# Определяем переменную Х через конкатенацию признаков с обработанными значениями NaN
Xs_train = pd.concat([Ds_train[num_feat].fillna(train_median),      # обрабатываем пустые значения, если таковые имеются
                     Ds_train[num_feat + cat_feat].isnull().astype(np.int8).add_suffix('_NaN'),
                     dummy_s_train_q], axis=1)

Xs_test = pd.concat([Ds_test[num_feat].fillna(train_median),
                     Ds_test[num_feat + cat_feat].isnull().astype(np.int8).add_suffix('_NaN'),
                     dummy_s_test_q], axis=1)



In [92]:
# Стандартиззируем выборку численных признаков путем удаления среднего и масштабирования до единичной дисперсии
scaler = StandardScaler()
scaler.fit(Xs_train[num_feat])

Xs_train[num_feat] = scaler.transform(Xs_train[num_feat])
Xs_test[num_feat] = scaler.transform(Xs_test[num_feat])

# Xs_train.shape, ys_train.shape

In [93]:
# Займемся непосредственым обученем стекинга 
# Финальное решение в стеке обучающихся моделей принимает логистическая регрессия  

classifier = StackingClassifier(
    [
        ('lr', LogisticRegression()),
        ('dt', DecisionTreeClassifier()),
        ('knn', KNeighborsClassifier())
    ],
LogisticRegression())
classifier.fit(Xs_train, ys_train)

C:\Users\zlatt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\zlatt\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
C:\Users\zlatt\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the da

C:\Users\zlatt\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:880: RuntimeWarning: Number of classes in training fold (465) does not match total number of classes (530). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  RuntimeWarning)


StackingClassifier(cv=None,
                   estimators=[('lr',
                                LogisticRegression(C=1.0, class_weight=None,
                                                   dual=False,
                                                   fit_intercept=True,
                                                   intercept_scaling=1,
                                                   l1_ratio=None, max_iter=100,
                                                   multi_class='auto',
                                                   n_jobs=None, penalty='l2',
                                                   random_state=None,
                                                   solver='lbfgs', tol=0.0001,
                                                   verbose=0,
                                                   warm_start=False)),
                               ('dt',
                                DecisionTreeClassifier(ccp_alpha=0.0,
                                

In [95]:
classifier.score(Xs_test, ys_test)


0.00684931506849315